## 1. 解释一下 GBDT 算法的过程

GBDT (Gradient Boosting Decision Tree)，全名叫梯度提升决策树，使用的是 **Boosting** 的思想。

### 1.1 Boosting 思想

Boosting 方法训练基分类器时采用串行的方式，各个基分类器之间有依赖。它的基本思路是将基分类器层层叠加，每一层在训练的时候，对前一层基分类器分错的样本，给予更高的权重。测试时，根据各层分类器的结果的加权得到最终结果。 

Bagging 与 Boosting 的串行训练方式不同，Bagging 方法在训练过程中，各基分类器之间无强依赖，可以进行并行训练。

### 1.2 GBDT 原来是这么回事

GBDT 的原理很简单，就是所有弱分类器的结果相加等于预测值，然后下一个弱分类器去拟合误差函数对预测值的残差(这个残差就是预测值与真实值之间的误差)。当然了，它里面的弱分类器的表现形式就是各棵树。

举一个非常简单的例子，比如我今年30岁了，但计算机或者模型 GBDT 并不知道我今年多少岁，那GBDT咋办呢？

- 它会在第一个弱分类器（或第一棵树中）随便用一个年龄比如 20 岁来拟合，然后发现误差有 10 岁；
- 接下来在第二棵树中，用 6 岁去拟合剩下的损失，发现差距还有 4 岁；
- 接着在第三棵树中用 3 岁拟合剩下的差距，发现差距只有 1 岁了；
- 最后在第四课树中用 1 岁拟合剩下的残差，完美。
- 最终，四棵树的结论加起来，就是真实年龄 30 岁（实际工程中，gbdt是计算负梯度，用负梯度近似残差）。

**为何 GBDT 可以用用负梯度近似残差呢？**

回归任务下，GBDT 在每一轮的迭代时对每个样本都会有一个预测值，此时的损失函数为均方差损失函数，

$l(y_i, y^i) = \frac{1}{2}(y_i - y^i)^2$

那此时的负梯度是这样计算的

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/16.gif?raw=true)

所以，当损失函数选用均方损失函数是时，每一次拟合的值就是（真实值 - 当前模型预测的值），即残差。此时的变量是 $y^i$，即“当前预测模型的值”，也就是对它求负梯度。

**训练过程**

简单起见，假定训练集只有 4 个人：A,B,C,D，他们的年龄分别是 14, 16, 24, 26。其中 A、B 分别是高一和高三学生；C, D 分别是应届毕业生和工作两年的员工。如果是用一棵传统的回归决策树来训练，会得到如下图所示结果：

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/17.png?raw=true)

现在我们使用 GBDT 来做这件事，由于数据太少，我们限定叶子节点做多有两个，即每棵树都只有一个分枝，并且限定只学两棵树。我们会得到如下图所示结果：

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/18.png?raw=true)

在第一棵树分枝和图1一样，由于 A,B 年龄较为相近，C,D 年龄较为相近，他们被分为左右两拨，每拨用平均年龄作为预测值。

- 此时计算残差（残差的意思就是：A 的实际值 - A 的预测值 = A 的残差），所以A的残差就是实际值 14 - 预测值 15 = 残差值 -1。
- 注意，A 的预测值是指前面所有树累加的和，这里前面只有一棵树所以直接是 15，如果还有树则需要都累加起来作为 A 的预测值。

然后拿它们的残差 -1、1、-1、1 代替 A B C D 的原值，到第二棵树去学习，第二棵树只有两个值 1 和 -1，直接分成两个节点，即 A 和 C 分在左边，B 和 D 分在右边，经过计算（比如 A，实际值 -1 - 预测值 -1 = 残差 0，比如 C，实际值 -1 - 预测值 -1 = 0），此时所有人的残差都是 0。残差值都为 0，相当于第二棵树的预测值和它们的实际值相等，则只需把第二棵树的结论累加到第一棵树上就能得到真实年龄了，即每个人都得到了真实的预测值。

换句话说，现在 A,B,C,D 的预测值都和真实年龄一致了。Perfect！

- A: 14 岁高一学生，购物较少，经常问学长问题，预测年龄 A = 15 – 1 = 14
- B: 16 岁高三学生，购物较少，经常被学弟问问题，预测年龄 B = 15 + 1 = 16
- C: 24 岁应届毕业生，购物较多，经常问师兄问题，预测年龄 C = 25 – 1 = 24
- D: 26 岁工作两年员工，购物较多，经常被师弟问问题，预测年龄 D = 25 + 1 = 26

所以，GBDT 需要将多棵树的得分累加得到最终的预测得分，且每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差。

## 2. 梯度提升和梯度下降的区别和联系是什么？ 

下表是梯度提升算法和梯度下降算法的对比情况。可以发现，两者都是在每 一轮迭代中，利用损失函数相对于模型的负梯度方向的信息来对当前模型进行更新，只不过在梯度下降中，模型是以参数化形式表示，从而模型的更新等价于参数的更新。而在梯度提升中，模型并不需要进行参数化表示，而是直接定义在函 数空间中，从而大大扩展了可以使用的模型种类。

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/19.jpg?raw=true)

## 3. **GBDT**的优点和局限性有哪些？ 

### 3.1 优点

1. 预测阶段的计算速度快，树与树之间可并行化计算。
2. 在分布稠密的数据集上，泛化能力和表达能力都很好，这使得 GBDT 在 Kaggle 的众多竞赛中，经常名列榜首。 
3. 采用决策树作为弱分类器使得 GBDT 模型具有较好的解释性和鲁棒性，能够自动发现特征间的高阶关系。

### 3.2 局限性

1. GBDT 在高维稀疏的数据集上，表现不如支持向量机或者神经网络。
2. GBDT 在处理文本分类特征问题上，相对其他模型的优势不如它在处理数值特征时明显。 
3. 训练过程需要串行训练，只能在决策树内部采用一些局部并行的手段提高训练速度。 

## 4. RF(随机森林) 与 GBDT 之间的区别与联系

**相同点**：

- 都是由多棵树组成，最终的结果都是由多棵树一起决定。
- RF 和 GBDT 在使用 CART 树时，可以是分类树或者回归树。

**不同点**：

- 组成随机森林的树可以并行生成，而 GBDT 是串行生成
- 随机森林的结果是多数表决表决的，而 GBDT 则是多棵树累加之和
- 随机森林对异常值不敏感，而 GBDT 对异常值比较敏感
- 随机森林是减少模型的方差，而 GBDT 是减少模型的偏差
- 随机森林不需要进行特征归一化。而 GBDT 则需要进行特征归一化

## 5. 代码实现